# 本文件说明
- 读取数据、预处理数据

# 基本设置

In [19]:
import xlrd

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor_circ

from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 一些函数

In [3]:
def predict_right(folder, file_list):
    '''预测正确数据'''
    right_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
    #         tmp_data = tmp_data[tmp_data['R_W'] == 'Right']
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#             tmp_data = tmp_data[tmp_data['备注'] != '删除']
    #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
#             print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            right_data = pd.concat([right_data, tmp_data], axis = 0)    
    return right_data

In [4]:
def correct_wrong_data(folder, file_list):
    '''预测错误修正后数据'''
    correct_wrong_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
    #         tmp_data = tmp_data[tmp_data['R_W'] == 'Wrong']
            print('去删除前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data[tmp_data['备注'] != '删除']
    #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
            print('去删除后： ', tmp_data.shape, file_name, sheet_name)

            tmp_data['人工判断'] = tmp_data.apply(lambda x:1 if x['label'] == x['备注'] else 0, axis = 1)
            tmp_data['算法判断'] = tmp_data.apply(lambda x:1 if x['predict_label'] == x['备注'] else 0, axis = 1)
            print('人工误判率： %0.2f'%(1 - tmp_data['人工判断'].sum() / tmp_data.shape[0]),
                  '算法误判率： %0.2f'%(1 - tmp_data['算法判断'].sum() / tmp_data.shape[0]))
            print()

            correct_wrong_data = pd.concat([correct_wrong_data, tmp_data], axis = 0)
    return correct_wrong_data

In [5]:
def get_new_data(folder, file_list):
    '''新补充数据'''
    new_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
            print(tmp_data.columns)
            tmp_data = tmp_data[['label', 'title', 'content']]
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data.dropna(subset = ['label'], axis = 0)
            print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            new_data = pd.concat([new_data, tmp_data], axis = 0)    
    return new_data

In [6]:
def get_PR_plot_data(feedback_data, date, plot_data):
    y_pred_class = feedback_data[feedback_data['date'] == date]['predict_label'].tolist()
    y_test = feedback_data[feedback_data['date'] == date]['label'].tolist()
    print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
    print(metrics.classification_report(y_test, y_pred_class))
    print('confusion_matrix: ')
    print( metrics.confusion_matrix(y_test, y_pred_class))
    print()       
    
    classification_report = metrics.classification_report(y_test, y_pred_class)
    lines = classification_report.split('\n')
    # plot_data = []
    for index, line in enumerate(lines[2 : len(lines)]):
        t = line.strip().split()
        if len(t) < 2: continue    
        if (index + 2) < (len(lines) - 2):
            t = [float(v) if i > 0 else v for i, v in enumerate(t)]
            plot_data.append(t + [date])
        else :
            t = [float(v) if i > 2 else v for i, v in enumerate(t) ]
            plot_data.append(['avg/total'] + t[3:] + [date])    
            
    return plot_data

In [7]:
def get_feedback_data(folder, file_list):
    '''模型预测后反馈数据'''
    feedback_data = pd.DataFrame()
    label_list = []
    sum_list = []
    right_list = []
    plot_data = []
    for file_name in file_list:
        print('  ----    ', file_name)
        date = file_name.split('_')[0]
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:   
            tmp_data = pd.read_excel(currentPath, sheet_name)    
            tmp_data = tmp_data[tmp_data['label'] != '删除']
            tmp_data_1 = tmp_data[tmp_data['label'] == sheet_name]

            label_list.append(sheet_name)
            sum_list.append(tmp_data.shape[0])
            right_list.append(tmp_data_1.shape[0])
            print('类别：', sheet_name, '总数：', tmp_data.shape[0], 
                  '正确数', tmp_data_1.shape[0], 
                  '正确比例', tmp_data_1.shape[0]/tmp_data.shape[0])
            
            if 'predict_label' in tmp_data_1.columns:
                tmp_data_1 = tmp_data[['predict_label', 'label', 'title', 'content']]
            else :
                tmp_data_1 = tmp_data[['label', 'title', 'content']]
                if contain_ch(sheet_name) :
                    tmp_data_1['predict_label'] = sheet_name
                else :
                    tmp_data_1['predict_label'] = ''
                
            tmp_data_1['date'] = date
            print('去空值前： ', tmp_data_1.shape, file_name, sheet_name)
            tmp_data_1 = tmp_data_1.dropna(subset = ['label'], axis = 0)
            print('去空值后： ', tmp_data_1.shape, file_name, sheet_name)
            feedback_data = pd.concat([feedback_data, tmp_data_1], axis = 0)
            print() 
                        
        plot_data = get_PR_plot_data(feedback_data, date, plot_data) 
            
    PR_data = feedback_data[['date', 'predict_label', 'label']]
    feedback_data = feedback_data.drop('date', axis = 1)
    feedback_data = feedback_data.drop('predict_label', axis = 1)
    PR_plot_data  = pd.DataFrame(plot_data, columns = ['label', 'precision', 'recall', 
                                                    'f1-score', 'support', 'date'])     
    
    return feedback_data, PR_data, PR_plot_data

In [8]:
def pre_save(pre_func, data, save_folder):
    print(data.shape)
    print('save_folder: ', save_folder)
    
    titles = pre_func(data['title'].tolist())
    print('title num: ', len(titles))
    save_filename = save_folder + 'corpus/titles.txt'
    fid = open(save_filename, "w+", encoding='UTF-8')
    for line in titles:
        fid.write(line + '\n')
    fid.close()  
    
#     print(len(data['content'].tolist()))
    contents = pre_func(data['content'].tolist())
    print('content num: ', len(contents))
    print(contents[0])
    # contents = [re.sub(r'[a-z]*', '', x) for x in contents]
    # print(len(contents))
    # print(contents[:2])
    coprus_save_filename = save_folder + 'corpus/contents.txt'
    f = open(coprus_save_filename, "w+", encoding='UTF-8')
    for line in contents:
        f.write(line + '\n')
    f.close()    

    label = data['label'].tolist()
    print('label num: ', len(label))
    coprus_save_filename = save_folder + 'corpus/labels.txt'
    f = open(coprus_save_filename, "w+", encoding='UTF-8')
    for line in label:
        f.write(str(line) + '\n')
    f.close()

    data.to_excel(save_folder + 'corpus/title_content_label.xlsx', index = False)
    

# 交通部：1 二分类

## 读取数据：0 不相关 1 相关

In [9]:
folder = 'cor_data/20190102'

### 预测正确数据

In [15]:
# file_list_1 = ['190102交通相关-4244条.xlsx',]
# circ_raw_class_1 = predict_right(folder, file_list_1)

# circ_raw_class_1['label'] = circ_raw_class_1['label'].apply(lambda x:x.strip())    
# circ_raw_class_1 = circ_raw_class_1[['label', 'raw_title', 'raw_content']]
# circ_raw_class_1.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_1.shape)
# circ_raw_class_1.head()

In [11]:
# circ_raw_class_1['label'].value_counts()

### 预测错误修正后数据

In [12]:
# file_list_2 = ['circ_class_sel_20180827-1.xlsx','circ_class_sel_20180827-2.xlsx']

# circ_raw_class_2 = correct_wrong_data(folder, file_list_2)
# circ_raw_class_2['label'] = circ_raw_class_2['label'].apply(lambda x:x.strip())  
# circ_raw_class_2 = circ_raw_class_2[['备注', 'raw_title', 'raw_content']]
# circ_raw_class_2.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_2.shape)
# circ_raw_class_2.head()

In [13]:
# circ_raw_class_2['label'].value_counts()

### 新补充数据

In [25]:
file_list_3 = ['190102交通相关-4244条.xlsx','190102（1228导出数据标注）.xlsx'
               ] # 多个 sheet

circ_raw_class_3 = get_new_data(folder, file_list_3)

# circ_raw_class_3['label'] = circ_raw_class_3['label'].apply(lambda x:x.strip())    
print('总数：', circ_raw_class_3.shape)
circ_raw_class_3.head()

Index(['title', 'title 2', 'content', 'url', 'site_name', '倾向性', 'label'], dtype='object')
去空值前：  (4244, 3) 190102交通相关-4244条.xlsx 交通语料
去空值后：  (4244, 3) 190102交通相关-4244条.xlsx 交通语料
Index(['title', 'url', 'content', '入库时间', 'label'], dtype='object')
去空值前：  (1594, 3) 190102（1228导出数据标注）.xlsx 入库数据|最大ID（3800825）
去空值后：  (1594, 3) 190102（1228导出数据标注）.xlsx 入库数据|最大ID（3800825）
总数： (5838, 3)


,label,title,content
0,1,注意在合肥这8条道路乱停放共享单车将受罚-新华网安徽频道,记者日前从合肥市庐阳区城管委获悉，庐阳区在划定北一环路、长江中路两条路为首批共享单车...
1,1,高速路倒车不仅走错还要挨罚,高速公路上倒车是一个非常危险的举动。近日，高警局湘潭支队民警在沪昆高速东往西1080公里...
2,1,台铁出轨事故：200余人死伤1名陆配伤势较重,据新华社台北10月22日电 据台湾铁路局消息，截至22日10时，台铁6432次普悠玛列车...
3,1,注意合肥长江中路划定非机动车停放区域乱停车将被拖移-新华网安徽频道,近日，记者从庐阳区城管委获悉，为了解决停放区域标线不明显、共享单车等非机动车乱停乱放...
4,1,国庆假期长沙查处1.6万起交通违法无较大交通事故,"2018 年国庆期间，长沙交警紧紧围绕 "" 四个不发生和一个明显减少 "" 的安保目标，全..."


In [26]:
circ_raw_class_3['label'].value_counts()

1    4895
0     943
Name: label, dtype: int64

### 模型预测后反馈数据

In [17]:
# file_list_4 = ['20180904_circ_class_feedback.xlsx', '20180905_circ_class_feedback.xlsx',
#                '20180912_circ_class_feedback.xlsx','20180920_circ_class_feedback.xlsx',
#                '20180925_circ_class_feedback.xlsx', '20181008_circ_class_feedback.xlsx',
#                '20181019_circ_class_feedback.xlsx', '20181112_circ_class_feedback.xlsx'
#               ] # 多个 sheet

# circ_raw_class_4, PR_data, PR_plot_data = get_feedback_data(folder, file_list_4)

# circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
# print('总数：', circ_raw_class_4.shape)
# circ_raw_class_4.head()

In [18]:
# circ_raw_class_4['label'].value_counts()

#### PR

In [19]:
# y_pred_class = PR_data['predict_label'].tolist()
# y_test = PR_data['label'].tolist()
# print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
# print(metrics.classification_report(y_test, y_pred_class))
# print('confusion_matrix: ')
# print( metrics.confusion_matrix(y_test, y_pred_class))
# # PR_data.head()

In [20]:
# PR_plot_data
# PR_plot_data = PR_plot_data.reset_index('date')

In [21]:
# PR_plot_data[PR_plot_data['label'] == 'avg/total'][[ 'date', 'precision', 'recall' ,'f1-score']].plot(x = 'date')
# PR_plot_data[PR_plot_data['label'] == 'avg/total']

In [22]:
# P_data = PR_plot_data[PR_plot_data['label'] != 'avg/total'][ ['date', 'label', 'precision']]
# P_data.set_index(['date','label']).unstack('label').plot(figsize = (15,6))
# P_data.set_index(['date','label']).unstack('label')

In [23]:
# P_data = PR_plot_data[PR_plot_data['label'] != 'avg/total'][ ['date', 'label', 'recall']]
# P_data.set_index(['date','label']).unstack('label').plot(figsize = (15,6))
# P_data.set_index(['date','label']).unstack('label')

In [24]:
# P_data = PR_plot_data[PR_plot_data['label'] != 'avg/total'][ ['date', 'label', 'f1-score']]
# P_data.set_index(['date','label']).unstack('label').plot(figsize = (15,6))
# P_data.set_index(['date','label']).unstack('label')

### 补录与预警数据

In [25]:
# file_list_5 = ['20180917_circ_class_feedback.xlsx'
#               ] 
# add_warn_data = pd.DataFrame()
# for file_name in file_list_5:
#     currentPath  = '{0}/{1}'.format(folder, file_name)
#     excel = xlrd.open_workbook(currentPath)
#     sheet_names = [sheet.name for sheet in excel.sheets()]
#     for sheet_name in sheet_names:   
#         tmp_data = pd.read_excel(currentPath, sheet_name)    
#         tmp_data = tmp_data[tmp_data['label'] != '删除']
#         add_warn_data = pd.concat([add_warn_data, tmp_data], axis = 0)

# circ_raw_class_5 = add_warn_data[['label', 'title', 'content']]

In [26]:
# from sklearn import metrics

# y_pred_class = add_warn_data['predict_label']
# y_test = add_warn_data['label']
# print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
# print(metrics.classification_report(y_test, y_pred_class))
# print('confusion_matrix: ')
# print( metrics.confusion_matrix(y_test, y_pred_class))

In [27]:
# circ_raw_class_5['label'].value_counts()

## 合并数据

In [28]:
# file_list_4 = ['20181019_circ_class_feedback.xlsx'] # 多个 sheet
# circ_raw_class_4, PR_data, PR_plot_data = get_feedback_data(folder, file_list_4)
# circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
# print('总数：', circ_raw_class_4.shape)
# circ_raw_class_4.head()

In [27]:
# circ_raw_class = pd.concat([circ_raw_class_1, circ_raw_class_2,
#                           circ_raw_class_3, circ_raw_class_4,
#                            circ_raw_class_5], 
#                          axis = 0)

circ_raw_class = circ_raw_class_3
circ_raw_class['title_content'] = circ_raw_class['title'].astype(str) + '。' + circ_raw_class['content'].astype(str)
# circ_raw_class['label'] = circ_raw_class['label'].apply(lambda x:x.strip())
circ_raw_class['index'] = range(circ_raw_class.shape[0])
print('去重前： ', circ_raw_class.shape)
circ_raw_class = circ_raw_class.drop_duplicates(subset = 'title')
print('去重后： ', circ_raw_class.shape)
circ_raw_class = circ_raw_class.drop_duplicates(subset = 'content')
print('去重后： ', circ_raw_class.shape)
circ_raw_class.head()

去重前：  (5838, 5)
去重后：  (5838, 5)
去重后：  (5628, 5)


,label,title,content,title_content,index
0,1,注意在合肥这8条道路乱停放共享单车将受罚-新华网安徽频道,记者日前从合肥市庐阳区城管委获悉，庐阳区在划定北一环路、长江中路两条路为首批共享单车...,注意在合肥这8条道路乱停放共享单车将受罚-新华网安徽频道。 记者日前从合肥市庐阳区城管...,0
1,1,高速路倒车不仅走错还要挨罚,高速公路上倒车是一个非常危险的举动。近日，高警局湘潭支队民警在沪昆高速东往西1080公里...,高速路倒车不仅走错还要挨罚。 高速公路上倒车是一个非常危险的举动。近日，高警局湘潭支队民警...,1
2,1,台铁出轨事故：200余人死伤1名陆配伤势较重,据新华社台北10月22日电 据台湾铁路局消息，截至22日10时，台铁6432次普悠玛列车...,台铁出轨事故：200余人死伤1名陆配伤势较重。 据新华社台北10月22日电 据台湾铁路局消...,2
3,1,注意合肥长江中路划定非机动车停放区域乱停车将被拖移-新华网安徽频道,近日，记者从庐阳区城管委获悉，为了解决停放区域标线不明显、共享单车等非机动车乱停乱放...,注意合肥长江中路划定非机动车停放区域乱停车将被拖移-新华网安徽频道。 近日，记者从庐阳...,3
4,1,国庆假期长沙查处1.6万起交通违法无较大交通事故,"2018 年国庆期间，长沙交警紧紧围绕 "" 四个不发生和一个明显减少 "" 的安保目标，全...",国庆假期长沙查处1.6万起交通违法无较大交通事故。 2018 年国庆期间，长沙交警紧紧围绕...,4


In [28]:
print(circ_raw_class.shape)
circ_raw_class.groupby('label')['title'].count()

(5628, 5)


label
0     925
1    4703
Name: title, dtype: int64

## 预处理

In [29]:
save_folder = 'cor_result/'
pre_save(pre_cor_circ.handle_contents, circ_raw_class, save_folder)

(5628, 5)
save_folder:  cor_result/
title num:  5628
content num:  5628
记者 日前 合肥市 庐阳区 城 管委 获悉 庐阳区 划定 北 环路 长江中路 两条路 首批 共享 单车 严管 路段 含 山路 环城路 条 道路 共享 单车 禁停 路段 禁停 路段 发现 乱 停放 共享 单车 庐阳区 城 管委 督促 单车 企业 分钟 清运 拖移 处置 后期 用户 违停 收到 单车 平台 提醒 收取 调度 费 扣取 信用 分 影响 后续 用车 条 路段 为北 环路 长江中路 含 山路 环城路 宿州 路 可苑 路 茨 河路 义井路 一步 庐阳区 城 管委 督促 单车 企业 利用 电子 围栏 定位 技术 平台 显示 禁停 路段 庐阳区 城 管委 负 用户 执意 挪车 各家 单车 企业 收取 调度 费 扣取 信用 分 影响 后续 用车 希望 正面 清单 负面 清单 相结合 停放 管理模式 培养 市民 停车 习惯 维护 出行 环境 
label num:  5628
